In [1]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.1'
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://security.ubuntu.com/ubuntu bionic-securi

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-01-19 22:48:04--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.1s    

2021-01-19 22:48:05 (6.63 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()


In [50]:
# Read in Data from CSV files
from pyspark import SparkFiles
file1 = "county_data_cleaned.csv"
spark.sparkContext.addFile(file1)
county_df = spark.read.csv(SparkFiles.get("county_data_cleaned.csv"), sep=",", header=True, inferSchema=True)

In [51]:
# Show data
county_df.show()

+---------+-----+--------------------+-----------------------+-------------+-------------+--------------------+-----------------+-------+------+--------+--------+--------+--------+-----+-----+------+-----+-------+-------+------------+-------+------------+
|   county|state|votes16_Donald_Trump|votes16_Hillary_Clinton|total_votes16|total_votes20|votes20_Donald_Trump|votes20_Joe_Biden|  cases|deaths|TotalPop|     Men|   Women|Hispanic|White|Black|Native|Asian|Pacific| Income|IncomePerCap|Poverty|Unemployment|
+---------+-----+--------------------+-----------------------+-------------+-------------+--------------------+-----------------+-------+------+--------+--------+--------+--------+-----+-----+------+-----+-------+-------+------------+-------+------------+
|Abbeville|   SC|              6742.0|                 3712.0|      10724.0|      12433.0|              8215.0|           4101.0|  805.0|  17.0| 24788.0| 12044.0| 12744.0|     1.3| 68.9| 27.6|   0.1|  0.3|    0.0|35254.0|     19234.

In [52]:
# Read in Data from CSV files
from pyspark import SparkFiles
file2 = "Covid_data.csv"
spark.sparkContext.addFile(file2)
covid_df = spark.read.csv(SparkFiles.get("Covid_data.csv"), sep=",", header=True, inferSchema=True)

In [53]:
#Show Covid Data
covid_df.show()

+---------+-----+-------+------+--------+
|   county|state|  cases|deaths|TotalPop|
+---------+-----+-------+------+--------+
|Abbeville|   SC|  805.0|  17.0| 24788.0|
|   Acadia|   LA| 3182.0| 102.0| 62607.0|
| Accomack|   VA| 1227.0|  19.0| 32840.0|
|      Ada|   ID|17451.0| 181.0|435117.0|
|    Adair|   IA|  222.0|   1.0|  7192.0|
|    Adair|   KY|  517.0|  22.0| 19304.0|
|    Adair|   MO|  578.0|   0.0| 25437.0|
|    Adair|   OK|  855.0|  11.0| 22136.0|
|    Adams|   CO|15538.0| 263.0|487850.0|
|    Adams|   IA|   87.0|   1.0|  3785.0|
|    Adams|   ID|   65.0|   2.0|  3946.0|
|    Adams|   IL| 2223.0|  25.0| 66787.0|
|    Adams|   IN|  704.0|   8.0| 35018.0|
|    Adams|   MS| 1122.0|  45.0| 31583.0|
|    Adams|   ND|   91.0|   0.0|  2368.0|
|    Adams|   NE|  922.0|  15.0| 31564.0|
|    Adams|   OH|  270.0|   6.0| 27926.0|
|    Adams|   PA| 1184.0|  31.0|101589.0|
|    Adams|   WA|  935.0|  11.0| 19261.0|
|    Adams|   WI|  587.0|   5.0| 20111.0|
+---------+-----+-------+------+--

In [54]:
# Read in Data from CSV files
from pyspark import SparkFiles
file3 = "Demographics.csv"
spark.sparkContext.addFile(file3)
Demographics_df = spark.read.csv(SparkFiles.get("Demographics.csv"), sep=",", header=True, inferSchema=True)

In [55]:
# Show Demographics data
Demographics_df.show()

+---------+-----+--------+--------+-----+-----+------+-----+-------+
|   county|state|TotalPop|Hispanic|White|Black|Native|Asian|Pacific|
+---------+-----+--------+--------+-----+-----+------+-----+-------+
|Abbeville|   SC| 24788.0|     1.3| 68.9| 27.6|   0.1|  0.3|    0.0|
|   Acadia|   LA| 62607.0|     2.4| 77.5| 17.6|   0.1|  0.1|    0.0|
| Accomack|   VA| 32840.0|     8.8| 60.3| 28.3|   0.3|  0.7|    0.0|
|      Ada|   ID|435117.0|     7.9| 85.2|  1.2|   0.4|  2.6|    0.1|
|    Adair|   IA|  7192.0|     1.7| 96.6|  0.3|   0.0|  0.4|    0.0|
|    Adair|   KY| 19304.0|     1.8| 93.4|  3.6|   0.1|  0.1|    0.0|
|    Adair|   MO| 25437.0|     2.3| 90.5|  2.4|   0.2|  2.3|    0.1|
|    Adair|   OK| 22136.0|     6.4| 40.8|  0.3|  41.7|  0.6|    0.2|
|    Adams|   CO|487850.0|    39.3| 51.1|  3.0|   0.5|  3.8|    0.1|
|    Adams|   IA|  3785.0|     1.2| 96.8|  0.4|   0.1|  0.6|    0.0|
|    Adams|   ID|  3946.0|     3.3| 92.7|  0.3|   1.2|  0.0|    0.2|
|    Adams|   IL| 66787.0|     1.5

In [56]:
# Read in Data from CSV files
from pyspark import SparkFiles
file4 = "Election_Results16.csv"
spark.sparkContext.addFile(file4)
Election_Results16_df = spark.read.csv(SparkFiles.get("Election_Results16.csv"), sep=",", header=True, inferSchema=True)

In [57]:
# Show Election Results for 2016
Election_Results16_df.show()

+---------+-----+--------------------+-----------------------+-------------+
|   county|state|votes16_Donald_Trump|votes16_Hillary_Clinton|total_votes16|
+---------+-----+--------------------+-----------------------+-------------+
|Abbeville|   SC|              6742.0|                 3712.0|      10724.0|
|   Acadia|   LA|             21159.0|                 5638.0|      27386.0|
| Accomack|   VA|              8582.0|                 6737.0|      15755.0|
|      Ada|   ID|             93748.0|                75676.0|     195587.0|
|    Adair|   IA|              2456.0|                 1127.0|       3759.0|
|    Adair|   KY|              6637.0|                 1323.0|       8231.0|
|    Adair|   MO|              6019.0|                 3495.0|      10137.0|
|    Adair|   OK|              4753.0|                 1374.0|       6468.0|
|    Adams|   CO|             73807.0|                86471.0|     175125.0|
|    Adams|   IA|              1393.0|                  565.0|       2082.0|

In [58]:
# Read in Data from CSV files
from pyspark import SparkFiles
file5 = "Election_Results20.csv"
spark.sparkContext.addFile(file5)
Election_Results20_df = spark.read.csv(SparkFiles.get("Election_Results20.csv"), sep=",", header=True, inferSchema=True)

In [59]:
# Show Election Results from 2020
Election_Results20_df.show()

+---------+-----+-------------+--------------------+-----------------+
|   county|state|total_votes20|votes20_Donald_Trump|votes20_Joe_Biden|
+---------+-----+-------------+--------------------+-----------------+
|Abbeville|   SC|      12433.0|              8215.0|           4101.0|
|   Acadia|   LA|      28425.0|             22596.0|           5443.0|
| Accomack|   VA|      16938.0|              9172.0|           7578.0|
|      Ada|   ID|     259389.0|            130699.0|         120539.0|
|    Adair|   IA|       4183.0|              2917.0|           1197.0|
|    Adair|   KY|       8766.0|              7275.0|           1391.0|
|    Adair|   MO|      10337.0|              6391.0|           3705.0|
|    Adair|   OK|       7108.0|              5585.0|           1387.0|
|    Adams|   CO|     234599.0|             94874.0|         132951.0|
|    Adams|   IA|       2158.0|              1528.0|            590.0|
|    Adams|   ID|       2586.0|              1941.0|            591.0|
|    A

In [60]:
# Read in Data from CSV files
from pyspark import SparkFiles
file6 = "Gender.csv"
spark.sparkContext.addFile(file6)
Gender_df = spark.read.csv(SparkFiles.get("Gender.csv"), sep=",", header=True, inferSchema=True)

In [61]:
# Show Gender data
Gender_df.show()

+---------+-----+--------+--------+
|   county|state|     Men|   Women|
+---------+-----+--------+--------+
|Abbeville|   SC| 12044.0| 12744.0|
|   Acadia|   LA| 30433.0| 32174.0|
| Accomack|   VA| 16079.0| 16761.0|
|      Ada|   ID|217999.0|217118.0|
|    Adair|   IA|  3552.0|  3640.0|
|    Adair|   KY|  9632.0|  9672.0|
|    Adair|   MO| 12013.0| 13424.0|
|    Adair|   OK| 10987.0| 11149.0|
|    Adams|   CO|245840.0|242010.0|
|    Adams|   IA|  1870.0|  1915.0|
|    Adams|   ID|  2021.0|  1925.0|
|    Adams|   IL| 32698.0| 34089.0|
|    Adams|   IN| 17361.0| 17657.0|
|    Adams|   MS| 15680.0| 15903.0|
|    Adams|   ND|  1168.0|  1200.0|
|    Adams|   NE| 15981.0| 15583.0|
|    Adams|   OH| 13960.0| 13966.0|
|    Adams|   PA| 50055.0| 51534.0|
|    Adams|   WA|  9750.0|  9511.0|
|    Adams|   WI| 10770.0|  9341.0|
+---------+-----+--------+--------+
only showing top 20 rows



In [62]:
# Read in Data from CSV files
from pyspark import SparkFiles
file7 = "Wealth_distribution.csv"
spark.sparkContext.addFile(file7)
Wealth_distribution_df = spark.read.csv(SparkFiles.get("Wealth_distribution.csv"), sep=",", header=True, inferSchema=True)

In [63]:
# Show Wealth distribution data
Wealth_distribution_df.show()

+---------+-----+-------+-------+------------+------------+
|   county|state|Poverty| Income|IncomePerCap|Unemployment|
+---------+-----+-------+-------+------------+------------+
|Abbeville|   SC|   22.7|35254.0|     19234.0|         9.4|
|   Acadia|   LA|   21.5|40492.0|     21591.0|         8.9|
| Accomack|   VA|   19.8|42260.0|     24266.0|         5.4|
|      Ada|   ID|   11.8|60151.0|     31642.0|         4.3|
|    Adair|   IA|    9.5|49477.0|     28861.0|         3.0|
|    Adair|   KY|   21.5|36575.0|     18408.0|         6.2|
|    Adair|   MO|   26.2|38750.0|     21778.0|         5.5|
|    Adair|   OK|   30.8|33366.0|     16576.0|         5.5|
|    Adams|   CO|   12.2|64087.0|     27487.0|         5.1|
|    Adams|   IA|   11.2|49745.0|     27022.0|         4.2|
|    Adams|   ID|   13.8|42727.0|     24315.0|         8.7|
|    Adams|   IL|   13.1|48454.0|     27256.0|         5.5|
|    Adams|   IN|   18.9|48290.0|     21534.0|         5.2|
|    Adams|   MS|   34.2|30359.0|     17

In [28]:
# Join 2020 Election with Covid data
Covid_2020_joined= Election_Results20_df.join(covid_df, on= ["state", "county"], how= "left")
Covid_2020_joined.show()

+-----+---------+-------------+--------------------+-----------------+-------+------+--------+
|state|   county|total_votes20|votes20_Donald_Trump|votes20_Joe_Biden|  cases|deaths|TotalPop|
+-----+---------+-------------+--------------------+-----------------+-------+------+--------+
|   SC|Abbeville|      12433.0|              8215.0|           4101.0|  805.0|  17.0| 24788.0|
|   LA|   Acadia|      28425.0|             22596.0|           5443.0| 3182.0| 102.0| 62607.0|
|   VA| Accomack|      16938.0|              9172.0|           7578.0| 1227.0|  19.0| 32840.0|
|   ID|      Ada|     259389.0|            130699.0|         120539.0|17451.0| 181.0|435117.0|
|   IA|    Adair|       4183.0|              2917.0|           1197.0|  222.0|   1.0|  7192.0|
|   KY|    Adair|       8766.0|              7275.0|           1391.0|  517.0|  22.0| 19304.0|
|   MO|    Adair|      10337.0|              6391.0|           3705.0|  578.0|   0.0| 25437.0|
|   OK|    Adair|       7108.0|              5585.

In [47]:
# Join Demographics with Covid Data
Covid_Demo_joined= Demographics_df.join(covid_df, on= ["state", "county", "TotalPop"], how= "left")
Covid_Demo_joined.show()

+-----+---------+--------+--------+-----+-----+------+-----+-------+-------+------+
|state|   county|TotalPop|Hispanic|White|Black|Native|Asian|Pacific|  cases|deaths|
+-----+---------+--------+--------+-----+-----+------+-----+-------+-------+------+
|   SC|Abbeville| 24788.0|     1.3| 68.9| 27.6|   0.1|  0.3|    0.0|  805.0|  17.0|
|   LA|   Acadia| 62607.0|     2.4| 77.5| 17.6|   0.1|  0.1|    0.0| 3182.0| 102.0|
|   VA| Accomack| 32840.0|     8.8| 60.3| 28.3|   0.3|  0.7|    0.0| 1227.0|  19.0|
|   ID|      Ada|435117.0|     7.9| 85.2|  1.2|   0.4|  2.6|    0.1|17451.0| 181.0|
|   IA|    Adair|  7192.0|     1.7| 96.6|  0.3|   0.0|  0.4|    0.0|  222.0|   1.0|
|   KY|    Adair| 19304.0|     1.8| 93.4|  3.6|   0.1|  0.1|    0.0|  517.0|  22.0|
|   MO|    Adair| 25437.0|     2.3| 90.5|  2.4|   0.2|  2.3|    0.1|  578.0|   0.0|
|   OK|    Adair| 22136.0|     6.4| 40.8|  0.3|  41.7|  0.6|    0.2|  855.0|  11.0|
|   CO|    Adams|487850.0|    39.3| 51.1|  3.0|   0.5|  3.8|    0.1|15538.0|

In [31]:
# Join Wealth Distribution with Covid Data
Covid_Wealth_joined= Wealth_distribution_df.join(covid_df, on= ["state", "county"], how= "left")
Covid_Wealth_joined.show()

+-----+---------+-------+-------+------------+------------+-------+------+--------+
|state|   county|Poverty| Income|IncomePerCap|Unemployment|  cases|deaths|TotalPop|
+-----+---------+-------+-------+------------+------------+-------+------+--------+
|   SC|Abbeville|   22.7|35254.0|     19234.0|         9.4|  805.0|  17.0| 24788.0|
|   LA|   Acadia|   21.5|40492.0|     21591.0|         8.9| 3182.0| 102.0| 62607.0|
|   VA| Accomack|   19.8|42260.0|     24266.0|         5.4| 1227.0|  19.0| 32840.0|
|   ID|      Ada|   11.8|60151.0|     31642.0|         4.3|17451.0| 181.0|435117.0|
|   IA|    Adair|    9.5|49477.0|     28861.0|         3.0|  222.0|   1.0|  7192.0|
|   KY|    Adair|   21.5|36575.0|     18408.0|         6.2|  517.0|  22.0| 19304.0|
|   MO|    Adair|   26.2|38750.0|     21778.0|         5.5|  578.0|   0.0| 25437.0|
|   OK|    Adair|   30.8|33366.0|     16576.0|         5.5|  855.0|  11.0| 22136.0|
|   CO|    Adams|   12.2|64087.0|     27487.0|         5.1|15538.0| 263.0|48

In [32]:
# Join Election results 2016 with 2020
Elections_joined= Election_Results16_df.join(Election_Results20_df, on= ["state", "county"], how= "left")
Elections_joined.show()

+-----+---------+--------------------+-----------------------+-------------+-------------+--------------------+-----------------+
|state|   county|votes16_Donald_Trump|votes16_Hillary_Clinton|total_votes16|total_votes20|votes20_Donald_Trump|votes20_Joe_Biden|
+-----+---------+--------------------+-----------------------+-------------+-------------+--------------------+-----------------+
|   SC|Abbeville|              6742.0|                 3712.0|      10724.0|      12433.0|              8215.0|           4101.0|
|   LA|   Acadia|             21159.0|                 5638.0|      27386.0|      28425.0|             22596.0|           5443.0|
|   VA| Accomack|              8582.0|                 6737.0|      15755.0|      16938.0|              9172.0|           7578.0|
|   ID|      Ada|             93748.0|                75676.0|     195587.0|     259389.0|            130699.0|         120539.0|
|   IA|    Adair|              2456.0|                 1127.0|       3759.0|       4183.0|

In [41]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://election-results.cfpmzdtom7fa.us-east-1.rds.amazonaws.com:5432/postgres"
config = {"user":"postgres",
          "password": "Election1",
          "driver":"org.postgresql.Driver"}

In [44]:
# Write DataFrame to election_joined table in RDS
Elections_joined.write.jdbc(url=jdbc_url, table='Election_joined', mode=mode, properties=config)

In [45]:
# Write DataFrame to Covid_Wealth_joined table in RDS
Covid_Wealth_joined.write.jdbc(url=jdbc_url, table='Covid_Wealth_joined', mode=mode, properties=config)

In [48]:
# Write DataFrame to Covid_Demo_joined table in RDS
Covid_Demo_joined.write.jdbc(url=jdbc_url, table='Covid_Demo_joined', mode=mode, properties=config)

In [49]:
# Write DataFrame to Covid_2020_joined table in RDS
Covid_2020_joined.write.jdbc(url=jdbc_url, table='Covid_2020_joined', mode=mode, properties=config)